In [7]:
import torch
from rfm import LaplaceRFM,NTKModel
import wandb
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    # find GPU memory in GB, keeping aside 1GB for safety
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory//1024**3 - 1 
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8

def fstar(X):
    return torch.cat([(X[:,0]>0)[:,None], 
	(X[:,1]<0.5)[:,None]], axis=1).float()

model = NTKModel( device=DEVICE, mem_gb=DEV_MEM_GB, diag=False)

n = 1000 # samples
d = 100  # dimension
c = 2    # classes

X_train = torch.randn(n, d, device=DEVICE)
X_test = torch.randn(n, d, device=DEVICE)
y_train = fstar(X_train)
y_test = fstar(X_test)


model.fit(
    (X_train, y_train), 
    (X_test, y_test), 
)

AttributeError: 'tuple' object has no attribute 'to'